<a href="https://colab.research.google.com/github/antopeza/Final-Project-Brainster/blob/main/Doc2Vec_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import os 

import re

import string
from string import punctuation


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import xgboost as xgb

import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, word_tokenize

import gensim
from gensim.parsing.preprocessing import remove_stopwords

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Embedding, LSTM, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

from imblearn.over_sampling import SMOTE
from collections import Counter

#from scikeras.wrappers import KerasClassifier

import pickle
from pickle import dump
from pickle import load

from imblearn.over_sampling import SMOTE
from collections import Counter

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive 
drive.mount('/drive')

Mounted at /drive


In [3]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
stops = set(stopwords.words('english'))
#print(stops)

In [5]:
stemmer =  PorterStemmer()
lemmer = WordNetLemmatizer()

In [6]:
# example
print(stemmer.stem('halves'))
print(lemmer.lemmatize('halves'))

halv
half


In [7]:
#df_raw = pd.read_csv(path)
df_raw = pd.read_csv('/drive/My Drive/MACHINE LEARNING/FINAL PROJECT/train.csv')

In [8]:
print('shape of df  = ', df_raw.shape)
print()
df_raw.head(2)

shape of df  =  (404290, 6)



,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0


In [8]:
df_raw[df_raw.isnull().any(axis=1)]

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0
363362,363362,493340,493341,NaN,My Chinese name is Haichao Yu. What English na...,0


In [8]:
df_raw = df_raw.fillna("")
df_raw.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [9]:
# randomly selected samples from df for the new df to perform operations
df = df_raw.sample(3100)
df = df[['question1', 'question2', 'is_duplicate']]
print(df.shape)

(3100, 3)


In [10]:
df_copy = df.copy()

# In case of split before normalization

In [97]:
y = df['is_duplicate']
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 0:-1], 
                                                                y, stratify = y, test_size = 0.2, random_state = 0)

print("Whole dataset: ", len(df))
print("X_train size =", len(X_train))
print("X_split size =", len(X_test))
print("y_train size =", len(y_train))
print("y_split size =", len(y_test))

Whole dataset:  3100
X_train size = 2480
X_split size = 620
y_train size = 2480
y_split size = 620


In [98]:
y_train_df = pd.DataFrame(y_train, columns=['is_duplicate'])
X_train = X_train.join(y_train_df, lsuffix='_left', rsuffix='_right')
X_train

,question1,question2,is_duplicate
66080,Can you suggest a reputed US based SEO service...,Can anyone give me a review of the MSG . My Da...,0
24056,"What is a lucrative career for an introverted,...",What is a lucrative career for someone who is ...,1
139620,What's your zodiac sign?,Are you sensitive? What's your zodiac sign?,0
226122,How old would Jesus be today?,How old is Jesus?,1
32413,What are the principles of federalism?,What are fundamental ethical principles?,0
...,...,...,...
107150,How was the 2G scam caught?,If anybody want to expose a biggest scams in g...,0
130687,How do I remove personal information from atla...,How do I remove my personal information from t...,1
314943,How is convict conditionning?,What is Convict conditioning?,1
89163,Why don't human lay eggs?,How does a chicken lay an egg within another e...,0


In [99]:
y_test_df = pd.DataFrame(y_test, columns=['is_duplicate'])
X_test = X_test.join(y_test_df, lsuffix='_left', rsuffix='_right')
X_test

,question1,question2,is_duplicate
394442,Can you provide me the list of the best hollyw...,What are the best top 10 movies of Hollywood e...,1
206906,Which is the best book to refer to completely ...,Which is the best book for understanding Kali ...,1
200493,Where is the best place to register the .it do...,Where is the best place to register an .is dom...,0
375357,Why do trained falcons wear blinders?,How do falcons get rid from honey bees?,0
381392,How is the value of gold calculated?,What is the value of gold today?,0
...,...,...,...
215428,What is the best launcher for Android?,What is best launcher for Android?,1
383811,Can I use NFL player images and logos on my we...,Who will win Super Bowl 51?,0
134865,Which country is better to immigrate from India?,Which are the easiest developed countries to i...,0
35963,"What's the difference between Google AdWords, ...",How can I use Google AdWords?,0


# Clean Data

In [11]:
def get_wordnet_pos(tag):
    if tag.startswith("N"):
        return wordnet.NOUN
    if tag.startswith("J"):
        return wordnet.ADJ
    if tag.startswith("V"):
        return wordnet.VERB
    if tag.startswith("R"):
        return wordnet.ADV
    return wordnet.NOUN

def clean_sentence(sentence, stopwords = False, lemmatize = False, stem = False):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)  # removing all caracters that are not alpha numeric
    #sentence = re.sub(r'\s{2,}', ' ', sentence
   
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    
    if stopwords:
        sentence = remove_stopwords(sentence)  
        
    if lemmatize:
        nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
        wn_tagged = map(lambda x: (x[0], get_wordnet_pos(x[1])), nltk_tagged)
        res_words = []
        for word, tag in wn_tagged:
            if tag is None:
                res_words.append(word)
            else :
                res_words.append(lemmer.lemmatize(word, tag))
        sentence = " ".join(res_words)
        
    if stem:
        sent_stemmed = ''
        for word in sentence.split():
            sent_stemmed += ' '+ stemmer.stem(word)
        sentence = sent_stemmed
   
    return sentence

def get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False):
    #sents1 = dfq[['question1']]  
    #sents2 = dfq[['question2']]
    cleaned_sentences1 = []
    cleaned_sentences2 = []
    
    for index, row in df.iterrows():
        # print (index, row)
        cleaned1 = clean_sentence(row['question1'], stopwords, lemmatize, stem) 
        cleaned2 = clean_sentence(row['question2'], stopwords, lemmatize, stem)

        cleaned_sentences1.append(cleaned1)
        cleaned_sentences2.append(cleaned2)
    return cleaned_sentences1, cleaned_sentences2

In [12]:
def to_df(q1cleaned, q2cleaned, df):
    '''cleaned sentences to df'''
    X_temp1 = pd.DataFrame()
    X_temp2 = pd.DataFrame()
    X_temp1['q1'] = pd.DataFrame(q1cleaned, index = df.index)
    X_temp2['q2'] = pd.DataFrame(q2cleaned, index = df.index)
    X_temp = pd.concat([X_temp1, X_temp2], axis = 1)
    X_temp['is_duplicate'] = df_raw['is_duplicate']
    return X_temp

In [101]:
# #Train part 
# # With stopwords lemmatized, not stemmed
# q1_sw, q2_sw = get_cleaned_senteces(X_train, stopwords = False, lemmatize = True, stem = False)
# assert len(q1_sw) == len(q2_sw)
# X_sw_train = to_df(q1_sw, q2_sw, X_train)
# X_sw_train.shape

(2480, 3)

In [13]:
#Entire data without split
# With stopwords lemmatized, not stemmed
q1_clean, q2_clean = get_cleaned_senteces(df_copy, stopwords = False, lemmatize = True, stem = False)
assert len(q1_clean) == len(q2_clean)
df_clean = to_df(q1_clean, q2_clean, df_copy)
df_clean.shape

(3100, 3)

In [18]:
df_clean.head(2)

,q1,q2,is_duplicate
192049,how exactly do catnip do it thing to cat,why how do catnip affect cat be it safe,1
307227,what book i should buy for learn english,what be the best book to learn english,0


In [102]:
# #Test part
# # With stopwords lemmatized, not stemmed
# q1_sw, q2_sw = get_cleaned_senteces(X_test, stopwords = False, lemmatize = True, stem = False)
# assert len(q1_sw) == len(q2_sw)
# X_sw_test = to_df(q1_sw, q2_sw, X_test)
# X_sw_test.shape

(620, 3)

In [119]:
# print('Value counts:\n', X_sw_train['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_train['is_duplicate'].value_counts()/X_sw_train['is_duplicate'].count()*100)
# X_sw_train['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of train labels', color = 'purple', size = 12)
# plt.show()

In [120]:
# print('Value counts:\n', X_sw_test['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_test['is_duplicate'].value_counts()/X_sw_test['is_duplicate'].count()*100)
# X_sw_test['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of test labels', color = 'purple', size = 12)
# plt.show()

In [121]:
# display (X_sw_train.head(2))
# display(X_sw_test.head(2))
# display(df_clean.head(2))

In [44]:
#X_sw_test[X_sw_test.isnull().any(axis=1)]

# *TOKENIZATION

In [14]:
def tokenization(text):
    tokens = re.split(' ',text)
    return tokens
def token(text):
    text = text.apply(lambda x: tokenization(x))
    return text

In [15]:
df_clean['q1'] = df_clean['q1'].pipe(token) 
df_clean['q2'] = df_clean['q2'].pipe(token) 
df_clean.head(2)

,q1,q2,is_duplicate
335301,"[star, war, creative, franchise, in, episode, ...","[star, war, creative, franchise, in, episode, ...",0
153236,"[do, it, hurt, your, eye, when, you, scroll, o...","[what, be, a, double, din, touch, screen, stereo]",0


# SECOND TRY

In [20]:
from gensim import corpora

dictionary_q1 = corpora.Dictionary(df_clean['q1'])
dictionary_q2 = corpora.Dictionary(df_clean['q2'])
print(dictionary_q1)
print(dictionary_q2)


Dictionary(5339 unique tokens: ['cat', 'catnip', 'do', 'exactly', 'how']...)
Dictionary(5117 unique tokens: ['affect', 'be', 'cat', 'catnip', 'do']...)


In [21]:
import pprint

In [141]:
#pprint.pprint(dictionary_q2.token2id)

In [22]:
new_doc = "Human computer interaction"
new_vec = dictionary_q1.doc2bow(new_doc.lower().split())
print(new_vec)

[(65, 1), (118, 1), (685, 1)]


In [23]:
bow_corpus_q1 = [dictionary_q1.doc2bow(text) for text in df_clean['q1']]
#bow_corpus_q2 = [dictionary_q2.doc2bow(text) for text in df_clean['q2']]

#pprint.pprint(bow_corpus)

In [23]:
print(bow_corpus_q1)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1)], [(12, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [(23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)], [(0, 1), (12, 1), (14, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)], [(6, 1), (12, 1), (14, 1), (32, 1), (40, 1), (41, 1), (42, 1)], [(12, 1), (14, 1), (24, 1), (39, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1)], [(12, 1), (14, 1), (16, 1), (39, 1), (49, 1), (50, 1), (51, 1), (52, 1)], [(12, 1), (14, 1), (16, 1), (39, 1), (49, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1)], [(12, 1), (14, 1), (24, 1), (39, 1), (47, 3), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1)], [(12, 1), (14, 1), (31, 1), (39, 1), (60, 1), (78, 1), (7

In [24]:
from gensim import models

# train the model
tfidf = models.TfidfModel(bow_corpus_q1)

# transform the "system minors" string
words = "system minors".lower().split()
print(tfidf[dictionary_q1.doc2bow(words)])

[(260, 1.0)]


In [25]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus_q1], num_features=12)

In [29]:
#query_document = 'system engineering'.split()
query_bow = [dictionary_q2.doc2bow(text) for text in df_clean['q2']]


In [ ]:
#query_bow = dictionary_q1.doc2bow( [dictionary_q2.doc2bow(text) for text in df_clean['q2']])
sims = index[tfidf[query_bow]]
#print(list(enumerate(sims)))

# DOC2VEC

In [19]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

q1_doc2vec = [TaggedDocument(d, [i]) for i, d in enumerate(df_clean['q1'])]
q1_doc2vec = [TaggedDocument(d, [i]) for i, d in enumerate(df_clean['q2'])]

In [21]:
X_sw_test[:2]

[TaggedDocument(words=['what', 'do', 'the', 'pakistani', 'think', 'about', 'india'], tags=[0]),
 TaggedDocument(words=['what', 'be', 'the', 'fast', 'way', 'to', 'get', 'skinny'], tags=[1])]

In [22]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [45]:
model[0]

array([-0.14346737,  0.07367106, -0.08637749, -0.16156931,  0.05178611,
       -0.12005797,  0.02251495,  0.12360012,  0.08418518,  0.0670051 ,
        0.04393555,  0.08234134, -0.0355239 , -0.07640567,  0.01656204,
        0.13114704,  0.02962174,  0.04989212, -0.0579648 ,  0.09834328,
        0.04894318,  0.10887893,  0.14718623, -0.01500463, -0.06308912,
       -0.09290814,  0.08607521,  0.01644696, -0.01485473,  0.04643139,
       -0.15119122,  0.07412843,  0.13147098,  0.01730472,  0.04320083,
       -0.12658969, -0.0311023 ,  0.05521949, -0.01891625, -0.11537345,
       -0.03366362,  0.05427921, -0.02984735,  0.2237516 ,  0.04548423,
        0.15428916,  0.00101094,  0.09230774, -0.00215006, -0.051616  ],
      dtype=float32)

In [23]:
model.build_vocab(X_sw_train)

In [43]:
X_sw_train[:2]

[TaggedDocument(words=['what', 'do', 'pakistanis', 'think', 'about', 'indian', 'people'], tags=[0]),
 TaggedDocument(words=['be', 'there', 'a', 'fast', 'way', 'to', 'get', 'skinny'], tags=[1])]

In [46]:
model_trained = model.train(X_sw_train, total_examples=model.corpus_count, epochs=model.epochs)

In [47]:
model_trained[0]

TypeError: ignored

In [25]:
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(vector)

[ 0.04499007  0.0864236  -0.03706289 -0.03678551 -0.00906008 -0.16343848
 -0.0572757   0.07899299  0.13047796  0.06490028  0.01566632  0.04707143
  0.03655515 -0.04805838  0.02168971 -0.07778078 -0.06048888 -0.06047449
 -0.0252022  -0.01896362  0.00943103  0.09845986  0.09864224 -0.04846334
 -0.08397691 -0.06900182  0.00543577  0.01739913 -0.01718567  0.08056425
 -0.13684279  0.01523853 -0.01372935 -0.09392435  0.06325174 -0.04838404
 -0.05876615  0.00459609 -0.06808765 -0.05414576 -0.00331771 -0.02095827
 -0.02220751  0.14215772  0.04132379  0.16295572 -0.02652618  0.05572951
 -0.01935194 -0.04076494]


#otro intento doc2vec

In [89]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

X_sw_train_tag1 = [TaggedDocument(d, [i]) for i, d in enumerate(X_cleaned['q1'])]
X_sw_train_tag2 = [TaggedDocument(d, [i]) for i, d in enumerate(X_cleaned['q2'])]


In [14]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

X_train_tag1 = [TaggedDocument(d, [i]) for i, d in enumerate(X_cleaned['q1'])]
X_train_tag2 = [TaggedDocument(d, [i]) for i, d in enumerate(X_cleaned['q2'])]

In [106]:
X_train_tag1[0:5]

[TaggedDocument(words='why do prophet muhammad marry 6 year old girl why dint he think to adopt her keep calm and answer', tags=[0]),
 TaggedDocument(words='how instagram be helpful', tags=[1]),
 TaggedDocument(words='be there an app that give you recipe base on what you have in the fridge', tags=[2]),
 TaggedDocument(words='what do the phrase shes get character mean', tags=[3]),
 TaggedDocument(words='what do lamb chop taste like', tags=[4])]

In [104]:
X_sw_test_tag1 = [TaggedDocument(d, [i]) for i, d in enumerate(X_sw_test['q1'])]
X_sw_test_tag2 = [TaggedDocument(d, [i]) for i, d in enumerate(X_sw_test['q2'])]

In [17]:
X_train_tagged = X_train_tag1 + X_train_tag2
#X_train_tagged

#X_test_tagged = X_sw_test_tag1 + X_sw_test_tag2


In [50]:
X_train_tagged

NameError: ignored

In [81]:
tag1 = pd.DataFrame(X_sw_train_tag1, index=X_cleaned.index)
tag2 = pd.DataFrame(X_sw_train_tag2, index=X_cleaned.index)
y = X_cleaned['is_duplicate']
X = pd.concat([tag1, tag2, y], axis=1)
X

,words,tags,words,tags,is_duplicate
28991,why do prophet muhammad marry 6 year old girl ...,[0],do prophet muhammad actually marry a six year ...,[0],1
4032,how instagram be helpful,[1],how do instagram work,[1],0
279529,be there an app that give you recipe base on w...,[2],be there an app or website that can suggest re...,[2],1
12645,what do the phrase shes get character mean,[3],what do this phrase mean he be president of th...,[3],0
378194,what do lamb chop taste like,[4],what do lamb taste like,[4],0
...,...,...,...,...,...
95497,how do it feel to be bisexual or a lesbian in ...,[3095],whenhow do you first realize your bisexuality,[3095],0
275875,what be some way to get rid of zit on my earlobe,[3096],how do i get rid of a zit on my ear,[3096],1
368403,what be the function of smooth endoplasmic ret...,[3097],what be the definition and function of smooth ...,[3097],0
204445,what be the difference between say i love you ...,[3098],love what be the difference between i love you...,[3098],0


In [82]:
y = X['is_duplicate']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y,test_size = 0.2, random_state = 0)

In [83]:
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)

(2480, 5)
(620, 5)
(620,)
(2480,)


In [76]:
def rf_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Random Forest Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    rf_param_grid = {'n_estimators':[200, 500, 800], 'min_samples_split':[5, 15], 'max_depth': [70, 150, None]}
    rf_clf = RandomForestClassifier(random_state = 0)
    
    rf_search = HalvingGridSearchCV(rf_clf, rf_param_grid, cv = splits, factor = 2, scoring = 'accuracy', 
                                        max_resources = 40, aggressive_elimination = True,verbose = 0)
    rf_search.fit(X_train, y_train)
    
    rf_model = rf_search.best_estimator_
    
    y_pred = rf_model.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    print (rf_model)
    #print('log loss', rf_loss_bow, '\nacc', rf_acc_bow)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, rf_model  

In [91]:
def rfClassifer(n_estimators, trainingSet, label, testSet):

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    forest = RandomForestClassifier(n_estimators)
    forest = forest.fit(trainingSet, label)
    result = forest.predict(testSet)

    return result

In [92]:
result = cfun.rfClassifer(3, X_train, y_train, X_test)

NameError: ignored

In [77]:
acc_rf, precision_rf, recall_rf, f1_score_rf, log_loss_rf, y_pred_rf, berf  = rf_clf(X_train, X_test, y_train, y_test)

ValueError: ignored

NameError: ignored

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
n_estimators = 10
max_depth = 3
random_forest = RandomForestClassifier(
    n_estimators=n_estimators, max_depth=max_depth, random_state=10
)
random_forest.fit(X_sw_train, y_train)

ValueError: ignored

In [41]:
y_train = df_clean['is_duplicate']

# Doc2Vec and similarity

In [18]:
model_doc2vec = Doc2Vec(vector_size =20, window=2, min_count=1, epochs=50)

In [19]:
model_doc2vec.build_vocab(X_train_tagged)

In [20]:
model_doc2vec.train(X_train_tagged, total_examples=model_doc2vec.corpus_count, epochs=model_doc2vec.epochs)

In [109]:
print(model_doc2vec.infer_vector(X_sw_test))

[ 0.00553507 -0.02057612 -0.00601494  0.01590716  0.00477619 -0.00957217
 -0.01354616  0.01898067 -0.00717559  0.00853975  0.00150559 -0.01450056
 -0.00628278 -0.02179538 -0.01993607 -0.02084691  0.00312932 -0.0190215
 -0.00581192  0.01894295]


In [90]:
#TEST THE MODEL ON X_sw_test

test = model_doc2vec.infer_vector(X_sw_test)
model_doc2vec.docvecs.most_similar(positive = [test])
#positive = List of sentences that contribute positively.


[(1607, 0.3674626350402832),
 (1454, 0.3508557975292206),
 (1876, 0.3459669351577759),
 (2316, 0.3180397152900696),
 (1868, 0.31006860733032227),
 (389, 0.3054264485836029),
 (1438, 0.28425800800323486),
 (1760, 0.272051066160202),
 (2109, 0.2675776481628418),
 (2054, 0.2670101821422577)]

In [103]:
X_sw_train.iloc[2054]

q1              what be the best self help book you have read ...
q2                   what be the top self help book i should read
is_duplicate                                                    1
Name: 106969, dtype: object

#random

# Models

In [110]:
labels = preprocessing.LabelEncoder()
# Convert continous y values to categorical
y_train_cat= labels.fit_transform(y_train)
print(y_train_cat)

#y_val_cat = labels.fit_transform(y_val)
#print(y_val_cat)

y_test_cat = labels.fit_transform(y_test)
print(y_test_cat)

[0 1 0 ... 1 0 0]
[1 1 0 0 0 0 1 1 1 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 1 0 0 1
 0 1 1 0 1 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 1
 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 0 1 0 1 1 0 1 1 1 0 1 0 0
 0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 1 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0
 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0
 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0 1 0
 1 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1
 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 1 0 0 1 1 0 0
 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 0 1 1 0 1 0 0
 0 1 1 0 0 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 1 0 1 1 0 0 1 0 0 0 1
 0 1 1 

# try

In [65]:
df_clean

,q1,q2,is_duplicate
35916,"[what, do, pakistanis, think, about, indian, p...","[what, do, the, pakistani, think, about, india]",1
100235,"[be, there, a, fast, way, to, get, skinny]","[what, be, the, fast, way, to, get, skinny]",1
269467,"[can, we, write, ssc, 10th, exam, without, joi...","[how, do, i, manage, my, proposal]",0
305636,"[what, do, the, french, mon, amour, mean, in, ...","[what, do, don, mean, in, french]",0
305191,"[what, be, the, best, way, to, reduce, thigh, ...","[how, do, i, reduce, my, thigh, size, man]",1
...,...,...,...
139298,"[how, do, i, use, instagram, on, my, computer]","[what, be, the, best, way, to, use, instagram]",0
48895,"[how, do, you, treat, a, hematoma, cause, when...","[why, be, hematoma, cause, when, draw, blood]",0
42561,"[what, be, the, food, you, can, eat, every, da...","[can, you, eat, same, thing, every, day, and, ...",1
16821,"[what, be, the, difference, between, procedura...","[what, be, the, difference, between, procedura...",1


In [16]:
# import
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
q1_tag = [TaggedDocument(d, [i]) for i, d in enumerate(df_clean['q1'])]
q2_tag = [TaggedDocument(d, [i]) for i, d in enumerate(df_clean['q2'])]
#tagged_data

In [17]:
q1_model = Doc2Vec(q1_tag, vector_size = 20, window = 2, min_count = 1, epochs = 100)
q2_model = Doc2Vec(q2_tag, vector_size = 20, window = 2, min_count = 1, epochs = 100)

In [31]:
q1_model[0]

array([ 0.5810266 ,  1.2083697 ,  0.9162264 , -0.0174998 , -0.15741944,
       -1.4726185 , -0.62223524,  0.5812599 ,  0.7438206 , -0.04712546,
       -0.8470433 ,  0.6188074 ,  0.6834843 ,  0.34443977, -0.66070366,
        0.66500264,  0.1121475 , -0.8982408 ,  0.18651539, -0.18247873],
      dtype=float32)

In [76]:
# #cosine similarity - to finish 
# test_doc = word_tokenize("What is the step by step guide to invest".lower())
# test_doc_vector = model.infer_vector(test_doc)
# model.docvecs.most_similar(positive = [test_doc_vector])


[(1555, 0.7962811589241028),
 (2760, 0.7377956509590149),
 (1810, 0.7373658418655396),
 (2765, 0.7061798572540283),
 (2729, 0.6963450312614441),
 (1806, 0.688534677028656),
 (1839, 0.685553789138794),
 (2233, 0.6750451326370239),
 (1854, 0.6687722206115723),
 (2190, 0.6662200093269348)]

In [78]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
q1 = [TaggedDocument(d, [i]) for i, d in enumerate(df_clean['q1'])]
q2 = [TaggedDocument(d, [i]) for i, d in enumerate(df_clean['q2'])]
#tagged_data

In [79]:
tagged_data = q1 + q2

In [85]:
model = Doc2Vec(tagged_data, vector_size = 20, window = 2, min_count = 1, epochs = 100)
model

In [81]:
# test_doc = word_tokenize("What is the step by step guide to invest".lower())
# test_doc_vector = model.infer_vector(test_doc)
# model.docvecs.most_similar(positive = [test_doc_vector])

[(1854, 0.8070089221000671),
 (42, 0.7843912839889526),
 (958, 0.779266357421875),
 (1183, 0.7622733116149902),
 (1839, 0.7594348192214966),
 (2007, 0.7579702138900757),
 (2760, 0.7547283172607422),
 (909, 0.7446797490119934),
 (2765, 0.7345895767211914),
 (1835, 0.732286810874939)]

#try svc

In [38]:
doc2vec_data = np.concatenate([q1_model[0], q2_model[0]])
doc2vec_data

array([ 0.5810266 ,  1.2083697 ,  0.9162264 , -0.0174998 , -0.15741944,
       -1.4726185 , -0.62223524,  0.5812599 ,  0.7438206 , -0.04712546,
       -0.8470433 ,  0.6188074 ,  0.6834843 ,  0.34443977, -0.66070366,
        0.66500264,  0.1121475 , -0.8982408 ,  0.18651539, -0.18247873,
        1.481154  ,  0.9385288 ,  0.7077114 ,  0.99032164, -0.01401499,
        0.03580603,  0.6962059 , -0.12830243,  0.4736636 , -0.28460807,
       -0.9491731 ,  0.17938454,  0.9872916 ,  0.70210314, -0.6939416 ,
        2.0760696 ,  0.17848212, -0.14690156,  1.9990524 ,  1.2086005 ],
      dtype=float32)

In [53]:
q1_arr = q1_model[0] 

for i in range(2999):
  arrT = q1_model[i+1]
  q1_arr = np.vstack([q1_arr, arrT])

print(q1_arr)

[[ 0.5810266   1.2083697   0.9162264  ... -0.8982408   0.18651539
  -0.18247873]
 [-0.07754783  0.57327145 -0.31971028 ... -0.22801255  1.1117383
   0.32918173]
 [-1.0053186  -0.52901155 -0.73507607 ... -0.2930685   0.610467
   0.5802145 ]
 ...
 [ 0.7498941  -0.4870866  -0.54625773 ... -0.93611306  0.41861948
  -0.7085733 ]
 [ 0.37088498 -0.1587948   0.32154924 ... -0.28210035  0.68039924
  -0.40266022]
 [-0.1347055  -0.20586298  0.2859487  ... -0.04282287  0.6394209
   0.29231715]]


In [50]:
from numpy import vstack

In [56]:
q1_df = pd.DataFrame(q1_arr)
q1_df.shape

(3000, 20)

In [58]:
q2_df = pd.DataFrame(q2_arr)
q2_df.shape

(3000, 20)

In [57]:
q2_arr = q2_model[0] 

for i in range(2999):
  arrT = q2_model[i+1]
  q2_arr = np.vstack([q2_arr, arrT])

print(q2_arr)

[[ 1.4811540e+00  9.3852878e-01  7.0771140e-01 ... -1.4690156e-01
   1.9990524e+00  1.2086005e+00]
 [ 4.8501411e-01 -1.8047783e-01  4.3742090e-01 ... -2.3784493e-01
   7.8135693e-01  3.2830998e-01]
 [-4.5950159e-02 -1.2032396e-01  2.2239833e-01 ...  3.5680946e-02
   8.3187902e-01 -1.2083004e-01]
 ...
 [ 1.0095431e+00 -1.2337197e+00 -8.0943614e-01 ... -2.1735021e-01
   1.9523093e-01 -4.6974349e-01]
 [ 1.6711544e-02 -1.3244456e-01 -2.8724796e-01 ... -9.9119626e-02
   5.5358613e-01  2.1528772e-01]
 [-1.9789477e-01 -3.3766299e-01 -1.4754611e-01 ... -4.0167922e-01
   2.7842870e-01 -1.3564983e-03]]


In [80]:
X_df = pd.concat([q1_df, q2_df], axis=1)
#df_X = pd.concat([X_df, y_df], axis=1)
X_df.shape

(3000, 40)

In [75]:
y = df_clean['is_duplicate']
y = np.array(y)
y_df = pd.DataFrame(y)
y_df

,0
0,0
1,0
2,0
3,1
4,0
...,...
3095,1
3096,0
3097,0
3098,0


In [84]:
df = X_df.join(y_df, lsuffix='_left', rsuffix='_right')
df

,0_left,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,0_right
0,0.581027,1.208370,0.916226,-0.017500,-0.157419,-1.472618,-0.622235,0.581260,0.743821,-0.047125,...,0.179385,0.987292,0.702103,-0.693942,2.076070,0.178482,-0.146902,1.999052,1.208601,0
1,-0.077548,0.573271,-0.319710,0.156488,-1.124121,-0.494646,0.209814,-0.245632,0.363747,1.211124,...,0.416060,0.834830,0.093918,-0.092923,-0.249957,-0.469445,-0.237845,0.781357,0.328310,0
2,-1.005319,-0.529012,-0.735076,-0.087035,0.656220,-0.940712,0.234486,-0.171080,0.701527,-0.418393,...,0.298512,0.261643,0.354140,-0.577934,-0.701905,0.071875,0.035681,0.831879,-0.120830,0
3,0.236661,-0.340131,0.474591,0.305714,-0.357004,-0.738526,0.123583,0.420354,-0.321707,0.342781,...,-1.139340,0.298838,0.307555,-0.253113,0.140093,0.129347,-0.380358,1.209907,-0.250593,1
4,0.253025,0.219740,-0.217731,-0.361429,-0.113995,-0.800904,0.112391,-0.371718,0.667065,0.249858,...,1.000945,-0.193619,0.709927,-0.765053,0.579880,-0.265695,-0.915273,0.360893,0.595944,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.501749,0.380240,0.004502,0.444053,-0.068857,0.119434,0.552885,0.994120,1.652993,-0.973230,...,-0.185514,0.676677,1.004518,-0.937552,-0.374168,0.306625,0.200239,1.416152,0.399804,0
2996,0.643296,0.253637,0.037233,0.322196,-0.028288,-1.029146,-0.631035,-0.318620,0.015862,-0.135705,...,0.048253,0.596732,0.522567,-0.264217,-0.171991,-0.212232,-0.398130,0.354515,0.228741,0
2997,0.749894,-0.487087,-0.546258,-1.720051,-0.247620,-0.538750,-0.818423,0.388426,0.368137,-0.801490,...,0.548187,0.017583,-0.267069,0.154549,0.436526,0.355640,-0.217350,0.195231,-0.469743,1
2998,0.370885,-0.158795,0.321549,0.193105,-0.041239,-0.654309,-0.426082,-0.762172,0.477382,0.582717,...,0.083249,0.422974,0.554576,-0.001491,0.607132,-0.624293,-0.099120,0.553586,0.215288,1


In [86]:
y= df['0_right']
X_train, X_test, y_train, y_test = train_test_split(df, y, stratify = y, test_size = 0.2, random_state = 0)

In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
#X, y = make_classification(random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)


RandomForestClassifier(max_depth=2, random_state=0)

In [93]:
from sklearn.model_selection import ShuffleSplit

In [96]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

In [91]:
def rf_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Random Forest Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    rf_param_grid = {'n_estimators':[200, 500, 800], 'min_samples_split':[5, 15], 'max_depth': [70, 150, None]}
    rf_clf = RandomForestClassifier(random_state = 0)
    
    rf_bow_search = HalvingGridSearchCV(rf_clf, rf_param_grid, cv = splits, factor = 2, scoring = 'accuracy', 
                                        max_resources = 40, aggressive_elimination = True,verbose = 0)
    rf_bow_search.fit(X_train, y_train)
    
    rf_bow_model = rf_bow_search.best_estimator_
    
    y_pred = rf_bow_model.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    print (rf_bow_model)
    #print('log loss', rf_loss_bow, '\nacc', rf_acc_bow)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, rf_bow_model

In [97]:
acc_rf, precision_rf, recall_rf, f1_score_rf, log_loss_rf, y_pred_rf, berf  = rf_clf(X_train, X_test, y_train, y_test)

RandomForestClassifier(min_samples_split=15, n_estimators=500, random_state=0)
